### Документация pyTelegramBotAPI на русском

- [`message_handler(...)`](https://pytba.readthedocs.io/ru/latest/sync_version/index.html#telebot.TeleBot.message_handler)

---


In [ ]:
from telebot import TeleBot
from telebot.types import Message
from io import BytesIO
from image_manager import init_board, draw_board
import db
import json


TOKEN = "ВАШ_ТОКЕН"  # TODO: Подставляем токен
TOKEN = "7749038016:AAEFTNfmM6ifUbrEjO4MtNxG_wTjTP1KvwI"  # TODO: Подставляем токен
is_white = False
game_id: str | None = None


bot = TeleBot(TOKEN)

db.init_db()
db.create_game_state_table()


def convert_to_bytes(board):
    """
    Преобразует изображение доски в байтовый поток для отправки пользователям.

    Параметры:
    - board: объект изображения доски.

    Возвращает:
    - байтовый поток (BytesIO) с изображением доски в формате PNG.
    """
    image_io = BytesIO()
    board.save(image_io, "PNG")
    image_io.seek(0)
    return image_io


@bot.message_handler(commands=["create_game"])
def create_game(message: Message):
    if not (message.from_user):
        return

    global game_id, is_white
    game_id = db.create_game(message.from_user.id)
    bot.send_message(message.from_user.id, "Вы создали игру с id: " + game_id)
    board, init_pos = init_board()
    db.create_game_state(game_id, "0", init_pos)
    bot.send_photo(message.chat.id, convert_to_bytes(board))
    is_white = True


@bot.message_handler(commands=["join_game"])
def join_game(message: Message):
    if not (message.from_user and message.text):
        return

    global game_id, is_white
    _, game_id = message.text.split(" ")
    if db.game_full(game_id):
        bot.send_message(message.chat.id, "Игра уже началась")
        return
    db.join_game(message.from_user.id, game_id)
    bot.send_message(message.chat.id, "Вы присоединились к игре с id: " + message.text)
    is_white = False
    white_user_id = db.get_user_id(game_id, "white")
    if white_user_id:
        bot.send_message(white_user_id[0], "К игре присоединился игрок " + message.from_user.first_name)


@bot.message_handler(commands=["move"])
def move(message: Message):
    if not (message.from_user and message.text and game_id):
        return

    msg = message.text.split(" ")
    if len(msg) != 3:
        bot.send_message(message.chat.id, "Неверный формат ввода должно быть /move A2 A4")
    user_black_id, user_white_id = db.user_ids(game_id)
    is_next_black, game_state = db.get_game_state(game_id)
    if bool(is_next_black) != (user_black_id == message.from_user.id):
        bot.send_message(message.chat.id, "Сейчас не ваш ход")
        return

    game_state = json.loads(game_state)

    _, move_from, move_to = message.text.split(" ")
    if move_from not in game_state:
        bot.send_message(message.chat.id, "В этой клетке нет фигуры")
        return
    game_state[move_to] = game_state[move_from]
    del game_state[move_from]
    db.update_game_state(game_id, int(not user_black_id == message.from_user.id), game_state)
    board = draw_board(game_state)
    bot.send_photo(user_white_id, convert_to_bytes(board))
    bot.send_photo(user_black_id, convert_to_bytes(board))


if __name__ == "__main__":
    bot.polling(none_stop=True)

TypeError: cannot unpack non-iterable PngImageFile object